In [1]:
!pip install redis

In [2]:
import pymongo
import gridfs
import codecs
from IPython.display import display, HTML, IFrame
import json
import redis

In [3]:
client = pymongo.MongoClient("mongodb://localhost:26000/")

## Collect Information about the database 

In [4]:
mydb = client["db"]

print(client.list_database_names())

print(mydb.list_collection_names())

['admin', 'config', 'db']
['beread', 'article', 'user', 'read', 'poprank', 'articlesci', 'bereadsci', 'readregion']


## Making a query as an example

In [5]:
# helper methods 
def count_col(col):
    count_users = mydb[col].count_documents({})
    print(col + " count = ", count_users)


In [6]:
mycol = mydb["user"]
user1 = mycol.find_one({'id':'u12'},{'name'})
print(user1)  
count_col("user")

{'_id': ObjectId('5fe17207a1aa426c4687f319'), 'name': 'user12'}
user count =  10000


## Query popular-rank

In [7]:
year = 2017
month = 12
category = 'science'

mycol = mydb["poprank"]
ranking = mycol.find_one({'year':year, 'month': month, 'category': category})
print('The top five articles for %s on year %s month %s'%(category, str(year), str(month)))
print(ranking['articleAidList'])  


The top five articles for science on year 2017 month 12
['4018', '7231', '3971', '4384', '8111']


## Insert a document 

- #### Insert to the user collection

In [8]:
new_user = {"timestamp": "1506328859788", "id": "u12", "uid": "25",
            "name": "user0", "gender": "female", "email": "email0",
            "phone": "phone0", "dept": "dept4", "grade": "grade1",
            "language": "zh", "region": "Beijing", "role": "role0",
            "preferTags": "tags27", "obtainedCredits": "17"}

user_id = mydb.user.insert_one(new_user).inserted_id
print(user_id)
count_col("user")

5fe1747d18d59b772f9f824d
user count =  10001


In [9]:
user_id = mydb.user.find(new_user)

In [10]:
with open("sample_users", "r") as sample_users:
    sample_users = sample_users.readlines()
new_users =  []

for user in sample_users:
    new_users.append(json.loads(user.strip()))
new_users

[{'timestamp': '1506328859002',
  'id': 'u28',
  'uid': '2',
  'name': 'user2',
  'gender': 'male',
  'email': 'email2',
  'phone': 'phone2',
  'dept': 'dept0',
  'grade': 'grade2',
  'language': 'zh',
  'region': 'Beijing',
  'role': 'role0',
  'preferTags': 'tags13',
  'obtainedCredits': '17'},
 {'timestamp': '1506328859002',
  'id': 'u39',
  'uid': '2',
  'name': 'user2',
  'gender': 'male',
  'email': 'email2',
  'phone': 'phone2',
  'dept': 'dept0',
  'grade': 'grade2',
  'language': 'zh',
  'region': 'Beijing',
  'role': 'role0',
  'preferTags': 'tags13',
  'obtainedCredits': '17'},
 {'timestamp': '1506328859002',
  'id': 'u25',
  'uid': '2',
  'name': 'user2',
  'gender': 'male',
  'email': 'email2',
  'phone': 'phone2',
  'dept': 'dept0',
  'grade': 'grade2',
  'language': 'zh',
  'region': 'Beijing',
  'role': 'role0',
  'preferTags': 'tags13',
  'obtainedCredits': '17'}]

In [11]:
mydb.user.insert_many(new_users)

In [12]:
count_col("user")

user count =  10004


## Updating Collections


In [13]:
myquery = { "id": "u25" }
newvalues = { "$set": { "name": "old_name" } }

print("Before update\n")
for doc in mydb.user.find(myquery, {"id", "name"}):
    print(doc)

print("\n\n")

print("After update\n")

mydb.user.update_many(myquery, newvalues)

for doc in mydb.user.find(myquery, {"id", "name"}):
    print(doc)

Before update

{'_id': ObjectId('5fe17207a1aa426c4687f32a'), 'id': 'u25', 'name': 'user25'}
{'_id': ObjectId('5fe1747d18d59b772f9f8250'), 'id': 'u25', 'name': 'user2'}



After update

{'_id': ObjectId('5fe17207a1aa426c4687f32a'), 'id': 'u25', 'name': 'old_name'}
{'_id': ObjectId('5fe1747d18d59b772f9f8250'), 'id': 'u25', 'name': 'old_name'}


## Read and display image from GridFS

In [14]:
# using caching on user table 

In [15]:
image_name = 'image_a0_0.jpg'

In [17]:
myclient = pymongo.MongoClient("mongodb://localhost:26000/") # connection
mydb = myclient["db"] # database name
fs = gridfs.GridFS(mydb) # connect to GridF
images = mydb['fs.files'].find({'filename':{'$regex':image_name}}) # find the image (input image name if neccesary)
image = fs.get(images[0]["_id"]) # get the first image
byte_data = image.read() 
base64_data = codecs.encode(byte_data, 'base64') # convert the byte data
img = base64_data.decode('utf-8') 
print(image_name)
display(HTML('<img src=\"data:image/png;base64,'+ img + '\" alt=\"image\" />')) # display in HTML


image_a0_0.jpg


## Caching using redis server

In [ ]:
r = redis.Redis()
user_id = "u900"
result = r.get(user_id)
if(result is None):
    
    result = mydb.user.find_one({'id': 'u900'}, {"name"})
    print(result)
    r.mset({user_id: result['name']})

In [ ]:
r.get(user_id)